## Example 3 - Meshes for Surface Process Models

This notebook introduces the `SurfaceProcessMesh` object, which builds upon the `TopoMesh` and introduces methods for finding the stream connectivity, catchment identification and handling local minima.

Here we demonstrate the stream flow components of the `SurfaceProcessMesh` 

> Note: Again, the API for the structured mesh is identical

#### Contents

- [Upstream area and stream power](#Upstream-area-and-stream-power)
- [Outflow analysis](#Outflow analysis)

In [1]:
from quagmire.tools import meshtools
from quagmire import TopoMesh, SurfaceProcessMesh

In [2]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [3]:
minX, maxX = -5.0, 5.0
minY, maxY = -5.0, 5.0,
dx, dy = 0.02, 0.02

x, y, simplices = meshtools.elliptical_mesh(minX, maxX, minY, maxY, dx, dy, random_scale=1.0)

DM = meshtools.create_DMPlex_from_points(x, y, bmask=None)

mesh = SurfaceProcessMesh(DM, downhill_neighbours=1)

print ("Triangulation has {} points".format(mesh.npoints))

0 - Delaunay triangulation 0.30847999999999987s
0 - Calculate node weights and area 0.006652000000000324s
0 - Find boundaries 0.00020199999999981344s
0 - cKDTree 0.008875999999999884s
0 - Construct neighbour cloud array 0.3161489999999998s
0 - Construct rbf weights 0.05278399999999994s
Triangulation has 62234 points


## Height field and Rainfall

We generate the usual cylindrically symmetry domed surface and add multiple channels incised along the boundary. Here is it interesting to leave out the random noise to see how discretisation error influence the surface flow paths.

The `SurfaceProcessMesh` stores a rainfall pattern that is used to compute the stream power assuming everything goes into the surface runoff it also records a sediment distribution pattern (etc).

In [4]:
radius  = np.sqrt((x**2 + y**2))
theta   = np.arctan2(y,x) + 0.1

height  = np.exp(-0.025*(x**2 + y**2)**2) + 0.25 * (0.2*radius)**4  * np.cos(5.0*theta)**2 
height  += 0.5 * (1.0-0.2*radius)
heightn  = height + np.random.random(height.size) * 0.01 # random noise

mesh.update_height(height)

# let's use a rainfall proportional to height (any choice is ok)
rainfall = height**2
mesh.update_surface_processes(rainfall, np.zeros_like(rainfall))


0 - Compute slopes 0.15411399999999986s
0 - Build downhill matrices 0.1316790000000001s
 - Upstream area 0.09476400000000007s


## Upstream area and stream power

Integrating information upstream is a key component of stream power laws that are often used in landscape evolution models. This is computed by multiple $\mathbf{D} \cdot \mathbf{A}_{\mathrm{upstream}}$ evaluations to accumulate the area downstream node-by-node on the mesh. This is handled by the `cumulative_flow(vector)` routine.

In [derivatives and hill slopes](#Derivatives-and-hill-slopes) we smoothed the entire landscape, however we can also target the RBF kernel to smooth just the streams:

```python
streamwise_smoothing(data, its, centre_weight=0.75)
```

where `its` indicates the number of iterations to smooth the field stream-wise. Increasing `its` smooths the field further afield upstream and downstream.

In [5]:
def compute_stream_power(mesh, slope, m=1, n=1):
    """
    Stream power law (q_s)
    """
    rainflux = mesh.rainfall_pattern_Variable.data
    rainfall = mesh.area * rainflux
    cumulative_rain = mesh.cumulative_flow(rainfall)    
    stream_power = cumulative_rain**m * slope**n
    stream_power[~mesh.bmask] = 0.0
    
    return stream_power

mesh.rainfall_pattern_Variable.data = rainfall


## 1. Smooth mesh 

mesh.update_height(height)
cumulative_rain = mesh.cumulative_flow(mesh.rainfall_pattern_Variable.data * mesh.area)
smooth_mesh_slope = mesh.slopeVariable.data.copy()

rbf_slope = mesh.rbf_smoother(mesh.slopeVariable.data)
sws_slope = mesh.streamwise_smoothing(mesh.slopeVariable.data, 3, 0.5)

stream_power = compute_stream_power(mesh, mesh.slopeVariable.data, m=1, n=1)

print("max stream power = {:.4f}, mean stream power = {:.4f}".format(stream_power.max(), stream_power.mean()))

# stream-wise smoothing
stream_power2 = compute_stream_power(mesh, sws_slope, m=1, n=1)

print("max stream power = {:.4f}, mean stream power = {:.4f}".format(stream_power2.max(), stream_power2.mean()))

# rbf smoothing
stream_power3 = compute_stream_power(mesh, rbf_slope, m=1, n=1)

print("max stream power = {:.4f}, mean stream power = {:.4f}".format(stream_power3.max(), stream_power3.mean()))

## 2. Rough mesh

mesh.update_height(heightn)

cumulative_rain_n = mesh.cumulative_flow(mesh.rainfall_pattern_Variable.data * mesh.area)

# mesh._construct_rbf_weights(delta=0.03)
# rbf_height = mesh.rbf_smoother(mesh.height)
# rbf_gradx, rbf_grady = mesh.derivative_grad(rbf_height)
# rbf_slope = np.hypot(rbf_gradx, rbf_grady)

stream_power_n = compute_stream_power(mesh, mesh.slopeVariable.data, m=1, n=1)
stream_power_n[~mesh.bmask] = 0.0


0 - Compute slopes 0.18838200000000027s
0 - Build downhill matrices 0.13188299999999975s
max stream power = 0.5402, mean stream power = 0.0108
max stream power = 1.6268, mean stream power = 0.0165
max stream power = 0.5419, mean stream power = 0.0108
0 - Compute slopes 0.13362000000000052s
0 - Build downhill matrices 0.13373499999999972s


AttributeError: 'SurfaceProcessMeshClass' object has no attribute 'rainfall_pattern'

In [ ]:
import lavavu

points = np.column_stack([mesh.tri.points, height])

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[600,600], near=-10.0)

tri1 = lv.triangles("triangles", wireframe=False)
tri1.vertices(points)
tri1.indices(mesh.tri.simplices)
tri1.values(smooth_mesh_slope, "slope")
tri1.values(mesh.slopeVariable.data, "rough mesh slope")
tri1.values(cumulative_rain, "cumulative rain")
tri1.values(cumulative_rain_n, "cumulative rain (rough)")
tri1.values(stream_power, "stream_power")
tri1.values(stream_power2, "sp: stream smoothed")
tri1.values(stream_power3, "sp: rbf smoothed")
tri1.values(stream_power_n, "sp: rough mesh")

tri1.colourmap("drywet")
tri1.colourbar()

lv.control.Panel()
lv.control.ObjectList()
tri1.control.List(["slope", 
                   "rough mesh slope",
                   "cumulative rain", 
                   "stream_power", 
                   "cumulative rain (rough)", 
                   "sp: stream smoothed", 
                   "sp: rbf smoothed", 
                   "sp: rough mesh" ], property="colourby", value="slope", command="redraw")
lv.control.show()

## Outflow analysis

The topography we have defined has multiple outflow points, which, in the analytic case, should be equal. If they differ, this is a result of the discretisation.

In [ ]:
outflow_nodes = mesh.identify_outflow_points()

print("{} outflow nodes:".format(len(outflow_nodes)))
print(outflow_nodes)


cum_rain = cumulative_rain[outflow_nodes]
cum_rain_n = cumulative_rain_n[outflow_nodes]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot bar graph of cumulative rain for each outflow point

fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(111, xlabel='outflow node', ylabel='cumulative rain')
ax1.bar(np.array(range(0,len(cum_rain))), width=0.4, height=cum_rain)
ax1.bar(np.array(range(0,len(cum_rain)))+0.5, width=0.4, height=cum_rain_n)
plt.show()


The downhill matrices are introduced in the next example, [Ex3-Multiple-downhill-pathways](./Ex3-Multiple-downhill-pathways.ipynb)